In [3]:
import teradatasql
import yaml
import sys
import os
import sqlparse
import pandas as pd



This will get the dbt config parameters

In [4]:
# get home directory
from os.path import expanduser
import pprint
home = expanduser("~")
# assume project is called dbt_clearscape
project = 'dbt_clearscape'
# read config file
with open(home + '/.dbt/profiles.yml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
# get project config
if project not in config:
    print('project not found in config file')
project_config = config[project]
# get target
target = project_config['target']
# get connection details
project_connection = project_config['outputs'][target]
#pprint.pprint(project_connection)


Connect to teradata

In [5]:
# connect to teradata
conn = teradatasql.connect(host=project_connection['server'], user=project_connection['username'], password=project_connection['password'])    

Test by loading dataframe

In [7]:
# Define your query
query = """SELECT databasename
  ,sum(currentperm)/1024**2(BIGINT) CurrentPerm 
  ,sum(maxperm)/1024**2(BIGINT) MaxPerm
  ,(sum(maxperm) - sum(currentperm))/1024**2(BIGINT)  AS FreePeam
FROM dbc.diskspacev 
GROUP BY 1
ORDER BY 4 DESC """

df = pd.read_sql(query, conn)
df.head(10)

C:\Users\cw171001\AppData\Local\Temp\ipykernel_3440\504767045.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,DatabaseName,CurrentPerm,MaxPerm,FreePeam
0,demo_user,18,28236,28217
1,DEMO_DemandForecast_db,19,2442,2423
2,DBC,190,1463,1273
3,modelops,0,953,952
4,SYSBAR,0,610,610
5,SYSLIB,21,610,588
6,SYSUDTLIB,2,488,485
7,TDBCMgmt,0,476,476
8,val,57,476,419
9,External_AP,0,305,305
